In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q

In [2]:
client = Elasticsearch('https://67a7441549120daa2dbeef8ac4f5bb2e.us-east-1.aws.found.io:9243')

//anaconda/lib/python2.7/site-packages/elasticsearch/connection/http_urllib3.py:70: UserWarning: Connecting to 67a7441549120daa2dbeef8ac4f5bb2e.us-east-1.aws.found.io using SSL with verify_certs=False is insecure.
  'Connecting to %s using SSL with verify_certs=False is insecure.' % host)


In [4]:
client.ping()

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


True

In [5]:
client.indices.get_mapping()

//anaconda/lib/python2.7/site-packages/urllib3/connectionpool.py:769: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


{u'plotly_datasets': {u'mappings': {u'ebola_2014': {u'properties': {u'Country': {u'type': u'string'},
     u'Lat': {u'type': u'float'},
     u'Lon': {u'type': u'float'},
     u'Month': {u'type': u'integer'},
     u'Value': {u'type': u'float'},
     u'Year': {u'type': u'integer'},
     u'index': {u'type': u'integer'}}}}},
 u'sample-data': {u'mappings': {u'test-type': {u'properties': {u'my-boolean-1': {u'type': u'boolean'},
     u'my-boolean-2': {u'type': u'boolean'},
     u'my-date-1': {u'format': u'strict_date_optional_time||epoch_millis',
      u'type': u'date'},
     u'my-date-2': {u'format': u'strict_date_optional_time||epoch_millis',
      u'type': u'date'},
     u'my-geo-point-1': {u'type': u'geo_point'},
     u'my-geo-point-2': {u'type': u'geo_point'},
     u'my-number-1': {u'type': u'long'},
     u'my-number-2': {u'type': u'long'},
     u'my-string-1': {u'type': u'string'},
     u'my-string-2': {u'type': u'string'}}}}}}

In [10]:
import pandas as pd
df = pd.DataFrame(pd.read_csv("/Users/Alexandre/Workstation/databases/26k-consumer-complaints.csv"))

In [11]:
df.head()

,Unnamed: 0,Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
0,0,1291006,Debt collection,NaN,Communication tactics,Frequent or repeated calls,TX,76119.0,2015-03-19,2015-03-19,"Premium Asset Services, LLC",In progress,Yes,NaN
1,1,1290580,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,TX,77479.0,2015-03-19,2015-03-19,Accounts Receivable Consultants Inc.,Closed with explanation,Yes,NaN
2,2,1290564,Mortgage,FHA mortgage,"Application, originator, mortgage broker",NaN,MA,2127.0,2015-03-19,2015-03-19,RBS Citizens,Closed with explanation,Yes,Yes
3,3,1291615,Credit card,NaN,Other,NaN,CA,92592.0,2015-03-19,2015-03-19,Navy FCU,In progress,Yes,NaN
4,4,1292165,Debt collection,Non-federal student loan,Cont'd attempts collect debt not owed,Debt resulted from identity theft,NaN,43068.0,2015-03-19,2015-03-19,Transworld Systems Inc.,In progress,Yes,NaN


In [13]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

,Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?
0,1291006,Debt collection,NaN,Communication tactics,Frequent or repeated calls,TX,76119.0,2015-03-19,2015-03-19,"Premium Asset Services, LLC",In progress,Yes,NaN
1,1290580,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt is not mine,TX,77479.0,2015-03-19,2015-03-19,Accounts Receivable Consultants Inc.,Closed with explanation,Yes,NaN
2,1290564,Mortgage,FHA mortgage,"Application, originator, mortgage broker",NaN,MA,2127.0,2015-03-19,2015-03-19,RBS Citizens,Closed with explanation,Yes,Yes
3,1291615,Credit card,NaN,Other,NaN,CA,92592.0,2015-03-19,2015-03-19,Navy FCU,In progress,Yes,NaN
4,1292165,Debt collection,Non-federal student loan,Cont'd attempts collect debt not owed,Debt resulted from identity theft,NaN,43068.0,2015-03-19,2015-03-19,Transworld Systems Inc.,In progress,Yes,NaN


In [14]:
help(client.indices.put_mapping)

Help on method put_mapping in module elasticsearch.client.indices:

put_mapping(*args, **kwargs) method of elasticsearch.client.indices.IndicesClient instance
    Register specific mapping definition for a specific type.
    `<http://www.elastic.co/guide/en/elasticsearch/reference/current/indices-put-mapping.html>`_
    
    :arg doc_type: The name of the document type
    :arg body: The mapping definition
    :arg index: A comma-separated list of index names the mapping should be
        added to (supports wildcards); use `_all` or omit to add the mapping
        on all indices.
    :arg allow_no_indices: Whether to ignore if a wildcard indices
        expression resolves into no concrete indices. (This includes `_all`
        string or when no indices have been specified)
    :arg expand_wildcards: Whether to expand wildcard expression to concrete
        indices that are open, closed or both., default 'open', valid
        choices are: 'open', 'closed', 'none', 'all'
    :arg ignore

In [17]:
mappings = dict()
mappings

{}

In [18]:
for name in df.columns:
  mappings[name] = {'type': 'string'}
  
mappings

{'Company': {'type': 'string'},
 'Company response': {'type': 'string'},
 'Complaint ID': {'type': 'string'},
 'Consumer disputed?': {'type': 'string'},
 'Date received': {'type': 'string'},
 'Date sent to company': {'type': 'string'},
 'Issue': {'type': 'string'},
 'Product': {'type': 'string'},
 'State': {'type': 'string'},
 'Sub-issue': {'type': 'string'},
 'Sub-product': {'type': 'string'},
 'Timely response?': {'type': 'string'},
 'ZIP code': {'type': 'string'}}

In [19]:
# upload the data to elasticsearch
client.indices.put_mapping(
  index='plotly_datasets',
  doc_type='consumer_complaints',    
  body={
    'properties': {
      {
       'Company': {'type': 'string'},
       'Company response': {'type': 'string'},
       'Complaint ID': {'type': 'integer'},
       'Consumer disputed?': {'type': 'string'},
       'Date received': {'type': 'date', 'format': 'strict_date_optional_time'},
       'Date sent to company': {'type': 'date', 'format': 'strict_date_optional_time'},
       'Issue': {'type': 'string'},
       'Product': {'type': 'string'},
       'State': {'type': 'string'},
       'Sub-issue': {'type': 'string'},
       'Sub-product': {'type': 'string'},
       'Timely response?': {'type': 'string'},
       'ZIP code': {'type': 'integer'}
      }
    }
  }
)

TypeError: unhashable type: 'dict'

In [9]:
bulk = []
for i in range(len(df)):
    bulk.append({'index': {'_index': 'plotly_datasets', '_type': 'ebola_2014', '_id': str(i)}})
    bulk.append(df.iloc[i].to_dict())

In [10]:
response = client.bulk(bulk, request_timeout=1000)

In [11]:
response.keys()

[u'items', u'errors', u'took']

In [12]:
response['errors']

False